In [1]:
import pandas as pd
import re
import persian
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
df = pd.read_excel('dataset/13981012-13981022_top_views.xlsx')

In [3]:
temp = df[df.text.duplicated()==False]

In [4]:
temp['original'] = temp['text'].copy(deep=False)
temp.head()

,created_at,text,type,views,peer_type,peer_username,peer_title,peer_participants,peer_about,fwd_type,fwd_username,fwd_title,fwd_participants,fwd_about,fwd_time,content_url,original
0,1398/10/12 11:47:05,دانلود و تماشای جدیدترین فیلم های سینمای ایرا...,photo,2383442,channel,namava_ir,Namava | نماوا,29126.0,📺 سرویس تماشای آنلاین فیلم و سریال 📺,NaN,NaN,NaN,NaN,NaN,NaN,NaN,دانلود و تماشای جدیدترین فیلم های سینمای ایرا...
1,1398/10/12 15:24:28,جشنواره زمستانی شروع شد معرفی و رونق ویژه کسب...,animated,1439671,channel,HiddenSender,a user,297947.0,The sender of the forwarded message chose to r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,جشنواره زمستانی شروع شد معرفی و رونق ویژه کسب...
2,1398/10/12 20:28:33,قطع و پایان همکاری استراماچونی با استقلال باشگ...,photo,1418642,channel,Nahavand_28,👑نهاوند28👑,36569.0,🌷تابع قوانین کشور🌷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,قطع و پایان همکاری استراماچونی با استقلال باشگ...
3,1398/10/12 20:28:33,فرهاد مجیدی سرمربی استقلال شد فرهاد مجیدی به ع...,photo,1418642,channel,Nahavand_28,👑نهاوند28👑,36569.0,🌷تابع قوانین کشور🌷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,فرهاد مجیدی سرمربی استقلال شد فرهاد مجیدی به ع...
4,1398/10/12 23:13:39,موقع بیرون رفتن از خونه وسایل با ارزشتون رو ه...,photo,1418642,channel,Nahavand_28,👑نهاوند28👑,36569.0,🌷تابع قوانین کشور🌷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,موقع بیرون رفتن از خونه وسایل با ارزشتون رو ه...


In [5]:
stopwords = []
file = open('stopwords.txt', encoding = 'utf-8').read()
[stopwords.append(x) for x in file.split()]
stopwords = set(stopwords)

In [6]:
def remove_general_stopwords(text):
    text = str(text)
    filtered_tokens = [token for token in text.split() if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [7]:
def remove_eng_words(text):
    return re.sub(r'[a-zA-Z]+', '', str(text))

In [8]:
def remove_digit(text):
    return re.sub(r'\d+', '', str(text))

In [9]:
def convert_ar_chars(text):
    s = ''
    for word in text:
        s = s + persian.convert_ar_characters(word)  
    return s

In [10]:
def just_persian(text):
    return re.sub(r'[^آ-ی]', ' ', str(text))

In [11]:
temp.text = temp.text.apply(remove_general_stopwords)

In [12]:
temp.text = temp.text.apply(remove_eng_words)

In [13]:
temp.text = temp.text.apply(remove_digit)

In [14]:
temp.text = temp.text.apply(convert_ar_chars)

In [15]:
temp.text = temp.text.apply(just_persian)

In [16]:
temp.text = temp.text.apply(remove_general_stopwords)

In [17]:
w_count = temp.text.str.split(expand=True).stack().value_counts()

In [18]:
w_count[:50]

ایران       5028
سلیمانی     4187
آمریکا      3668
قاسم        2661
سردار       2410
عراق        2083
آمریکایی    1650
شهید        1648
ترامپ       1295
کشور        1236
حمله        1235
خبر         1191
تهران       1148
رئیس        1130
حاج         1129
اعلام       1079
منطقه       1049
روز          999
وزیر         979
هواپیمای     962
شهادت        959
مردم         953
انتقام       936
سپاه         913
پایگاه       899
سقوط         898
نیروهای      877
نظامی        843
سپهبد        814
تشییع        805
ترور         790
حضور         785
مراسم        774
استقلال      764
خارجه        721
کشته         706
پیکر         702
فرمانده      657
اوکراینی     647
اقدام        625
موشک         622
سخت          581
سازمان       580
پاسخ         557
کرمان        549
دولت         549
اسلامی       547
المللی       547
باشگاه       542
ایرانی       542
dtype: int64

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [20]:
# temp.fillna('',inplace=True)

In [21]:
X = temp.text

In [22]:
model = Pipeline([('vect', CountVectorizer(min_df=30, lowercase=False)),
                     ('tfidf', TfidfTransformer(sublinear_tf=True))])
XX = model.fit_transform(X)

In [23]:
model.named_steps['vect'].get_feature_names()

['آب',
 'آبان',
 'آبی',
 'آتش',
 'آثار',
 'آخر',
 'آدرس',
 'آرام',
 'آرامش',
 'آزاد',
 'آزادی',
 'آسمان',
 'آسمانی',
 'آسیا',
 'آسیب',
 'آشکار',
 'آغوش',
 'آل',
 'آلمان',
 'آماده',
 'آمادگی',
 'آمار',
 'آمریکا',
 'آمریکاست',
 'آمریکای',
 'آمریکایی',
 'آموزان',
 'آموزش',
 'آمیز',
 'آهن',
 'آور',
 'آورده',
 'آژانس',
 'آگاه',
 'آیت',
 'آید',
 'آینده',
 'ائتلاف',
 'ابتدای',
 'ابراز',
 'ابلاغ',
 'ابو',
 'ابومهدی',
 'اتحادیه',
 'اتخاذ',
 'اتصال',
 'اتفاق',
 'اتفاقات',
 'اتفاقی',
 'اتمی',
 'اتوبوس',
 'اجازه',
 'اجتماعی',
 'اجرای',
 'اجرایی',
 'اجساد',
 'احترام',
 'احتمال',
 'احتمالا',
 'احتمالی',
 'احساس',
 'احضار',
 'احمد',
 'اخبار',
 'اخبارفوری',
 'اختصاص',
 'اختصاصی',
 'اختیار',
 'اخراج',
 'اداره',
 'ادعا',
 'ادعای',
 'ارائه',
 'اربیل',
 'ارتباط',
 'ارتباطات',
 'ارتش',
 'ارتفاع',
 'ارز',
 'ارزش',
 'ارزیابی',
 'ارشد',
 'اروپا',
 'اروپایی',
 'ازدحام',
 'اساس',
 'اساسی',
 'اسامی',
 'استاد',
 'استان',
 'استرالیا',
 'استراماچونی',
 'استعفا',
 'استعفای',
 'استقبال',
 'استقرار',
 'استقلال',
 'است

In [24]:
tfidf_df = pd.DataFrame(XX.toarray(),
                        columns=model.named_steps['vect'].get_feature_names())

In [25]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=1)
kmeans.fit(tfidf_df)
temp['labels'] = kmeans.labels_

In [26]:
temp.labels.value_counts()

2    3541
6    1731
3    1361
4     727
1     707
5     647
8     468
7     388
9     130
0     115
Name: labels, dtype: int64

In [27]:
temp[temp.labels == 8]

,created_at,text,type,views,peer_type,peer_username,peer_title,peer_participants,peer_about,fwd_type,fwd_username,fwd_title,fwd_participants,fwd_about,fwd_time,content_url,original,labels
982,1398/10/12 18:21:59,ساعاتی ترافیک سابقه پرواز همزمان هواپیماهای تر...,photo,110616,channel,javanmardi77,AVATODAY - علی جوانمردی Ali Javanmardi77,305664.0,📝Independent journalist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,از ساعاتی پیش، ترافیک بی سابقه و پرواز همزمان...,8
1772,1398/10/13 22:15:39,تصویری حمله الاسد موشکهای بالستیک هدف واقعی مت...,photo,1272616,channel,Nahavand_28,👑نهاوند28👑,36569.0,🌷تابع قوانین کشور🌷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,تصویری که از حمله به عین_الاسد منتشر شده و گف...,8
1793,1398/10/13 15:38:00,کنایه مایکل مور کارگردان مستندساز آمریکایی ایر...,photo,575031,channel,gizmiztel,کانال گیزمیز 💯,1277895.0,﷽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,کنایه مایکل مور، کارگردان و مستندساز آمریکایی...,8
1849,1398/10/13 23:35:22,تایید وقوع انفجارهای متعدد پایگاه هوایی الاسد ...,photo,378404,channel,akharinkhabar,کانال آخرین خبر,3240634.0,🔺در کوتاه ترین زمان به روزترین باشید,NaN,NaN,NaN,NaN,NaN,NaN,NaN,تایید وقوع انفجارهای متعدد در پایگاه هوایی عی...,8
1967,1398/10/13 15:31:18,پست اینستاگرامی مایکل مور کارگردان مستندساز آم...,photo,321769,channel,akharinkhabar,کانال آخرین خبر,3240634.0,🔺در کوتاه ترین زمان به روزترین باشید,NaN,NaN,NaN,NaN,NaN,NaN,NaN,پست اینستاگرامی مایکل مور، کارگردان و مستندسا...,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18010,1398/10/21 10:40:02,ستاد پروازهای جنگی آمریکا حساسیت پدافند هوایی ...,text,283589,channel,bbcpersian,BBCPersian,1096633.0,کانال رسمی بخش فارسی بی‌بی‌سی در تلگرام,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ستاد کل افزایش پروازهای جنگی آمریکا باعث حساس...,8
18030,1398/10/21 16:56:52,خبرنگار الاسد تلفات ایم خبرنگار گزارش تصویری پ...,text,281687,channel,akhbarefori,کانال خبر فوری,2183858.0,﷽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,خبرنگار CNN ما در عین الاسد حتما تلفات داده ا...,8
18148,1398/10/21 16:55:50,خبرنگار الاسد تلفات ایم خبرنگار گزارش تصویری پ...,text,264162,channel,Khabar_Fouri,خبرهای فوری / مهم🔖,1388035.0,فوری‌ترین و مهمترین اخبار ایران و جهان 📰,NaN,NaN,NaN,NaN,NaN,NaN,NaN,فوری خبرنگار CNN ما در عین الاسد حتما تلفات د...,8
18199,1398/10/21 17:24:37,مجلس حضور فرمانده سپاه غیرعلنی برگزار رسانه ای...,text,256904,channel,bbcpersian,BBCPersian,1096633.0,کانال رسمی بخش فارسی بی‌بی‌سی در تلگرام,NaN,NaN,NaN,NaN,NaN,NaN,NaN,مجلس با حضور فرمانده سپاه جلسه غیرعلنی برگزار...,8


In [28]:
dic = {
    0:'سیاست خارجی ،  ترور و وقایع  پیرامونی' ,
    1:'سیاست خارجی ،  ترور و وقایع  پیرامونی' ,
    2:'ترور و وقایع  پیرامونی' ,
    3:'انتقام سخت ، سیاست خارجی ،  ترور و وقایع  پیرامونی' ,
    4:'هواپیمای اوکراینی و وقایع پیرامونی',
    5:'حمله به سفارت امریکا ، سیاست خارجی ،  ترور و وقایع  پیرامونی' ,
    6:'0' , 
    7:'سیاست خارجی' , 
    8:'حوادث طبیعی', 
    9:'سیاست خارجی ،  ترور و وقایع  پیرامونی' ,
}

In [29]:
temp.labels = temp.labels.replace(dic)

In [30]:
# temp[temp.labels != '0'].to_excel('valid1.xlsx')